<a href="https://colab.research.google.com/github/catundchat/LLM_report/blob/main/STT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

语音识别代码示例，使用方法：
1. choose appropriate pretrained model and tokenizer
2. load audio file
3. run the demo to generate transcription

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import soundfile as sf

# 加载预训练的模型和分词器
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

# 读取音频文件
audio_file = "audio_file.wav"
speech, _ = sf.read(audio_file)

# 对音频进行分词
input_values = tokenizer(speech, return_tensors="pt").input_values

# 用模型进行预测
logits = model(input_values).logits

# 获取预测结果
predicted_ids = logits.argmax(-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

print("Transcription:", transcription)
